In [ ]:
import tensorflow as tf
import numpy as np,sklearn,os,cv2,pandas
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras import Model,layers


In [ ]:
df=pandas.read_csv("./sentiment-emotion-labelled_Dell_tweets.csv")
df.head()

In [ ]:
X,Y=df.Text,df.sentiment
Y.unique()

In [ ]:
Classes=['neutral', 'positive', 'negative']

In [ ]:
X.shape,Y.shape

In [ ]:
X=np.array(X)
Y=np.array(Y)

In [ ]:
new_df=pandas.DataFrame(zip(X,Y),columns=["X","Y"])
new_df.head()

In [ ]:
bert_preprocess = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
New_Y=[]
for i,j in enumerate(Y):
    label=-2
    for k,l in enumerate(Classes):
        if l==j:
            label=k
            New_Y.append(label)
New_Y=np.array(New_Y)
New_Y.shape,X.shape

In [ ]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X,New_Y,test_size=0.2, random_state=42,shuffle=True)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
X_train[0],y_train[0]

In [ ]:
textinput=layers.Input(shape=(),dtype=tf.string,name="text")
pre=bert_preprocess(textinput)
end=bert_encoder(pre)
# NN Layer
po=end["sequence_output"]  #["pooled_output"]
l=layers.Bidirectional(layers.GRU(128, return_sequences=True))(po)
l=layers.Bidirectional(layers.GRU(64, return_sequences=True))(l)
l=layers.Conv1D(64, 1, activation="relu", padding='same')(l)
l=layers.Flatten()(l)
l=layers.Dropout(0.2)(l)
l=layers.Dense(128,activation="relu",name="input")(l)
l=layers.Dense(64,activation="relu",name="in")(l)
l=layers.Dense(16,activation="relu",name="in1")(l)
l=layers.Dropout(0.2)(l)
l=layers.Dense(3,activation="softmax",name="output")(l)
Model=tf.keras.Model(inputs=textinput,outputs=[l])


In [ ]:
Model.compile("adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
Model.summary()

In [ ]:
hist=Model.fit(X_train,y_train,epochs=30,batch_size=80)

In [ ]:
Model.evaluate(X_test,y_test)

In [ ]:
Model.save("./Model_3_Bert_Uncase.h5")

In [ ]:
A=Model.predict(["you awake from a deep trance, step away from the phone to see your friends & family"])
a=np.argmax(A)
Classes[a],int(A[0][a]*100)